# Steam Scraper

In [ ]:
# imports to scrape
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.ui import Select

In [ ]:
# Read csv of all steam apps, taken from https://api.steampowered.com/ISteamApps/GetAppList/v2/ and converted to csv
steamapps = pd.read_csv("steamapps.csv")
steamapps = steamapps[steamapps['name'].notna()].reset_index(drop=True)
steamapps

In [ ]:
# Turn all appids to a readable text document
with open("appids.txt", 'a') as f:
    f.write(steamapps['appid'].to_string(header=False, index=False))

In [ ]:
# Read the appids file
with open("oops.txt", "r") as f:
    for i, line in enumerate(f):
        appids = f.readlines()

In [ ]:
# Manually check every steam page
url = "https://store.steampowered.com/app/"

driver = webdriver.Edge()
datasets = []

# Log in to steam
driver.get("https://store.steampowered.com/login/")

# Wait to manually login
time.sleep(10)

for app in appids:
    appid = app.strip("\n").replace(" ", "")
    try:
        driver.get(url+appid)
    except:
        print(appid)
    # Checks if the link redirects to steam home, if it does, skip it (it is not an available app)
    try:
        age = driver.find_element("id", "ageYear")
        drop=Select(age)
        drop.select_by_visible_text("2003")
        driver.find_element("id", "view_product_page_btn").click()
        driver.get(url+appid)
    except:
        pass
    if (driver.current_url != "https://store.steampowered.com/"):
        row_data = []
        # Use beautiful soup to get data:
        try:
            soup = BeautifulSoup(driver.page_source)
        except:
            print(appid)
        row_data.append(appid)
        # Get game name
        try:
            name = soup.find(class_="apphub_AppName")
            row_data.append(name.get_text())
        except:
            row_data.append("")
        # Get developer and publisher
        dev_list = ""
        try:
            dev = soup.find(id="developers_list").find_all("a")
            for x in dev:
                dev_list += x.get_text() + ","
        except:
            pass
        row_data.append(dev_list.strip(","))
        try:
            publisher = soup.find(class_="dev_row").find("a")
            row_data.append(publisher.get_text())
        except:
            row_data.append("")
        # Get price
        try:
            price = soup.find(class_="game_purchase_price price", attrs={"data-price-final": True})
            row_data.append(price.get_text().replace("\t", "").replace("\n", ""))
        except:
            row_data.append("$0.00")
        # Get release date
        try:
            release_date = soup.find(class_="release_date").find(class_="date")
            row_data.append(release_date.get_text())
        except:
            row_data.append("")
        # Get Number of reviews
        try:
            reviews = soup.find(class_="user_reviews_filter_menu").find_all(class_="user_reviews_count")
        except:
            pass
        # Total reviews
        try:
            row_data.append(reviews[0].get_text().replace("(", "").replace(")", ""))
        except:
            row_data.append("")
        # Positive reviews
        try:
            row_data.append(reviews[1].get_text().replace("(", "").replace(")", ""))
        except:
            row_data.append("")
        # Negative reviews
        try:
            row_data.append(reviews[2].get_text().replace("(", "").replace(")", ""))
        except:
            row_data.append("")
        
        # Get Genres
        genres_list = ""
        try:
            genres = soup.find(id="genresAndManufacturer").find("span").find_all("a")
            for x in genres:
                genres_list += x.get_text() + ","
        except:
            pass
        row_data.append(genres_list.strip(","))
        # Get popular tags
        tags_list = ""
        try:
            pop_tags = soup.find(class_="glance_tags popular_tags").find_all("a")
            for x in pop_tags:
                tags_list += x.get_text().replace("\t", "").replace("\n", "") + ","
        except:
            pass
        row_data.append(tags_list.strip(","))
        # Get categories
        categories_list = ""
        try:
            categories = soup.find(id="category_block").find_all(class_="label")
            for x in categories:
                categories_list += x.get_text().replace("\t", "").replace("\n", "") + ","
        except:
            pass
        row_data.append(categories_list.strip(","))
        # Get languages
        languages_list = ""
        try:
            languages = soup.find(class_="game_language_options").find_all(class_="ellipsis")
            for x in languages:
                languages_list += x.get_text().replace("\t", "").replace("\n", "") + ","
        except:
            pass
        row_data.append(languages_list.strip(","))
        # Get achievement data
        try:
            achievement = soup.find(id="achievement_block").find(class_="block_title")
            row_data.append(achievement.get_text().replace("\t", "").replace("\n", ""))
        except:
            row_data.append("")
        
        datasets.append(row_data)

        # Free memory
        try:
            del(appid)
        except:
            print("hi")
            pass
        try:
            del(age)
        except:
            pass
        try:
            del(drop)
        except:
            pass
        try:
            del(row_data)
        except:
            pass
        try:
            del(soup)
        except:
            pass
        try:
            del(name)
        except:
            pass
        try:
            del(dev_list)
        except:
            pass
        try:
            del(dev)
        except:
            pass
        try:
            del(publisher)
        except:
            pass
        try:
            del(price)
        except:
            pass
        try:
            del(release_date)
        except:
            pass
        try:
            del(reviews)
        except:
            pass
        try:
            del(genres_list)
        except:
            pass
        try:
            del(tags_list)
        except:
            pass
        try:
            del(genres)
        except:
            pass
        try:
            del(pop_tags)
        except:
            pass
        try:
            del(categories_list)
        except:
            pass
        try:
            del(categories)
        except:
            pass
        try:
            del(languages_list)
        except:
            pass
        try:
            del(achievement)
        except:
            pass
        try:
            del(row_data)
        except:
            pass
        
driver.close()

In [ ]:
df = pd.DataFrame(data=datasets).drop_duplicates()
df